In [1]:
import os
import zipfile
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from keras.optimizers import SGD
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
from PIL import Image
import numpy as np
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [9]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  train_datagen = ImageDataGenerator(rescale=1./255,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')

  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=32,
                                                      class_mode='categorical',
                                                      target_size=(32, 32))

  validation_datagen = ImageDataGenerator(rescale=1./255)

  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=32,
                                                                class_mode='categorical',
                                                                target_size=(32, 32))

  return train_generator, validation_generator

In [10]:
train_dir = r"E:\projects\personalProjects\CSE525\AlexNet_Implementation\data\MNIST\train"
validation_dir = r"E:\projects\personalProjects\CSE525\AlexNet_Implementation\data\MNIST\test"

In [11]:
train_generator, validation_generator = train_val_generators(train_dir, validation_dir)

Found 60000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [3]:
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [5]:
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [6]:
def ResNet34(shape = (32, 32, 3), classes = 10):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

In [7]:
model = ResNet34()

In [8]:
model.compile(optimizer=Adam(learning_rate=0.001),
                loss='categorical_crossentropy',
                metrics=['accuracy', 'AUC', 'FalseNegatives', 'FalsePositives', 'Precision', 'Recall', 'TrueNegatives','TruePositives'])

In [12]:
import time

In [13]:
start = time.time()
history = model.fit(train_generator,
                    epochs=15,
                    verbose=1,
                    validation_data=validation_generator)
stop = time.time()

Epoch 1/15
1875/1875 [==============================] - 486s 252ms/step - loss: 0.8783 - accuracy: 0.6926 - auc: 0.9549 - false_negatives: 23746.0000 - false_positives: 6812.0000 - precision: 0.8418 - recall: 0.6042 - true_negatives: 533188.0000 - true_positives: 36254.0000 - val_loss: 0.4033 - val_accuracy: 0.8732 - val_auc: 0.9869 - val_false_negatives: 1414.0000 - val_false_positives: 1072.0000 - val_precision: 0.8890 - val_recall: 0.8586 - val_true_negatives: 88928.0000 - val_true_positives: 8586.0000
Epoch 2/15
1875/1875 [==============================] - 145s 77ms/step - loss: 0.3261 - accuracy: 0.9014 - auc: 0.9911 - false_negatives: 7076.0000 - false_positives: 4365.0000 - precision: 0.9238 - recall: 0.8821 - true_negatives: 535635.0000 - true_positives: 52924.0000 - val_loss: 0.2386 - val_accuracy: 0.9293 - val_auc: 0.9936 - val_false_negatives: 776.0000 - val_false_positives: 586.0000 - val_precision: 0.9403 - val_recall: 0.9224 - val_true_negatives: 89414.0000 - val_true_pos

In [14]:
start = time.time()
model.evaluate(validation_generator, verbose=1)
stop = time.time()

313/313 [==============================] - 9s 30ms/step - loss: 0.0657 - accuracy: 0.9786 - auc: 0.9993 - false_negatives: 224.0000 - false_positives: 186.0000 - precision: 0.9813 - recall: 0.9776 - true_negatives: 89814.0000 - true_positives: 9776.0000


In [16]:
print(f'Inferencing time: {(stop-start)/60} minutes')

Inferencing time: 0.18391307592391967 minutes
